In [1]:
#Importing Libraries
!pip install pandas
!pip install numpy
!pip install tensorflow

In [52]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from pathlib import Path
from tensorflow.keras import models, layers

In [85]:
#Defining Directories of Data
training_data = Path(r"C:\Users\jakem\OneDrive\Desktop\School\Machine Learning\Final Project\train")
testing_data = Path(r"C:\Users\jakem\OneDrive\Desktop\School\Machine Learning\Final Project\test")
validation_data = Path(r"C:\Users\jakem\OneDrive\Desktop\School\Machine Learning\Final Project\validation")

In [86]:
train_filepaths = list(training_data.glob(r'**/*.jpg'))
testing_filepaths = list(testing_data.glob(r'**/*.jpg'))
validation_filepaths = list(validation_data.glob(r'**/*.jpg'))

In [87]:
str(train_filepaths[3])

'C:\\Users\\jakem\\OneDrive\\Desktop\\School\\Machine Learning\\Final Project\\train\\apple\\Image_17.jpg'

In [88]:
for i in train_filepaths:
    print(str(i).split("\\")[-2])

apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
apple
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
banana
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroot
beetroo

In [89]:
# Creating image dataset with labels
def create_dataset(path) -> pd.DataFrame:
    """
    This function takes in a path and returns a dataset with images and labels

    Args:
    path: filepath to dataset

    Returns:
    A pandas DataFrame
    """
    # Obtaining the labels from the file paths of the images
    label = [str(path[i]).split("\\")[-2] for i in range(0,len(path))]

    path = pd.Series(path, name='Path').astype(str)
    label = pd.Series(label, name='Label')

    # Combining into a dataframe
    data = pd.concat([path, label], axis=1)

    # Shuffling the data
    data = data.sample(frac=1)
    data = data.reset_index(drop=True)

    return data


In [90]:
#Creating my datasets from the filepaths
train_data = create_dataset(train_filepaths)
test_data = create_dataset(testing_filepaths)
val_data = create_dataset(validation_filepaths)

In [91]:
train_data

,Path,Label
0,C:\Users\jakem\OneDrive\Desktop\School\Machine...,onion
1,C:\Users\jakem\OneDrive\Desktop\School\Machine...,mango
2,C:\Users\jakem\OneDrive\Desktop\School\Machine...,onion
3,C:\Users\jakem\OneDrive\Desktop\School\Machine...,lettuce
4,C:\Users\jakem\OneDrive\Desktop\School\Machine...,pineapple
...,...,...
2850,C:\Users\jakem\OneDrive\Desktop\School\Machine...,beetroot
2851,C:\Users\jakem\OneDrive\Desktop\School\Machine...,spinach
2852,C:\Users\jakem\OneDrive\Desktop\School\Machine...,potato
2853,C:\Users\jakem\OneDrive\Desktop\School\Machine...,grapes


In [92]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [93]:
#Loading Training Images
train_images = train_generator.flow_from_dataframe(
    dataframe=train_data,
    x_col='Path',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16,
    shuffle=True,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

Found 2855 validated image filenames belonging to 36 classes.


In [94]:
#Loading Training Images
test_images = test_generator.flow_from_dataframe(
    dataframe=test_data,
    x_col='Path',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16,
    shuffle=True,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

Found 340 validated image filenames belonging to 36 classes.


In [95]:
#Loading Training Images
val_images = train_generator.flow_from_dataframe(
    dataframe=val_data,
    x_col='Path',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16,
    shuffle=True,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

Found 340 validated image filenames belonging to 36 classes.


In [96]:
#Loading in a pretrained model
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

In [98]:
#Defining Inpute from pretrained model
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

#Creating prediction layer with 36 food classes
outputs = tf.keras.layers.Dense(36, activation='softmax')(x)

pretrained_models = tf.keras.Model(inputs=inputs, outputs=outputs)

#Compiling and fitting model
pretrained_models.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

hist = pretrained_models.fit(
    train_images,
    validation_data=val_images,
    batch_size = 16,
    epochs=10,
)

Epoch 1/10
179/179 [==============================] - 122s 670ms/step - loss: 1.5595 - accuracy: 0.5741 - val_loss: 0.4309 - val_accuracy: 0.8647
Epoch 2/10
179/179 [==============================] - 114s 638ms/step - loss: 0.5462 - accuracy: 0.8256 - val_loss: 0.3243 - val_accuracy: 0.8971
Epoch 3/10
179/179 [==============================] - 109s 610ms/step - loss: 0.3062 - accuracy: 0.8876 - val_loss: 0.2504 - val_accuracy: 0.9206
Epoch 4/10
179/179 [==============================] - 111s 619ms/step - loss: 0.2011 - accuracy: 0.9306 - val_loss: 0.2119 - val_accuracy: 0.9353
Epoch 5/10
179/179 [==============================] - 111s 620ms/step - loss: 0.1296 - accuracy: 0.9555 - val_loss: 0.1648 - val_accuracy: 0.9500
Epoch 6/10
179/179 [==============================] - 111s 621ms/step - loss: 0.0957 - accuracy: 0.9632 - val_loss: 0.2054 - val_accuracy: 0.9647
Epoch 7/10
179/179 [==============================] - 111s 619ms/step - loss: 0.0712 - accuracy: 0.9762 - val_loss: 0.1780 -

In [61]:
#Defining Parameters for Custom Model
BATCH_SIZE = 16
IMAGE_LENGTH = 244
IMAGE_WIDTH = 244
RGB = 3

In [62]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    r"C:\Users\jakem\OneDrive\Desktop\School\Machine Learning\Final Project\train",
    shuffle=True,
    image_size=(IMAGE_LENGTH, IMAGE_WIDTH),
    batch_size = BATCH_SIZE
)

Found 3115 files belonging to 36 classes.


In [63]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    r"C:\Users\jakem\OneDrive\Desktop\School\Machine Learning\Final Project\test",
    shuffle=True,
    image_size=(IMAGE_LENGTH, IMAGE_WIDTH),
    batch_size = BATCH_SIZE
)

Found 359 files belonging to 36 classes.


In [64]:
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    r"C:\Users\jakem\OneDrive\Desktop\School\Machine Learning\Final Project\validation",
    shuffle=True,
    image_size=(IMAGE_LENGTH, IMAGE_WIDTH),
    batch_size = BATCH_SIZE
)

Found 351 files belonging to 36 classes.


In [66]:
class_names = train_dataset.class_names
class_names

['apple',
 'banana',
 'beetroot',
 'bell pepper',
 'cabbage',
 'capsicum',
 'carrot',
 'cauliflower',
 'chilli pepper',
 'corn',
 'cucumber',
 'eggplant',
 'garlic',
 'ginger',
 'grapes',
 'jalepeno',
 'kiwi',
 'lemon',
 'lettuce',
 'mango',
 'onion',
 'orange',
 'paprika',
 'pear',
 'peas',
 'pineapple',
 'pomegranate',
 'potato',
 'raddish',
 'soy beans',
 'spinach',
 'sweetcorn',
 'sweetpotato',
 'tomato',
 'turnip',
 'watermelon']

In [68]:
#Shuffling Data
test_dataset = test_dataset.shuffle(10000)
train_dataset = train_dataset.shuffle(10000)
val_dataset = val_dataset.shuffle(10000)

In [70]:
#Improving the performance by caching the images and prefecthing to properly utilize CPU 
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [71]:
#Normalizing the image size to between 0 and 1
sized_scaled_layer = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_LENGTH, IMAGE_WIDTH),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [72]:
#Creating a layer to apply effects to images so the model generalizes better
altered_layer = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [73]:
#Building the Custom Convolutional Neural Network
class_size = 36

model = models.Sequential([
    sized_scaled_layer,
    altered_layer,
    #Convolutional layer with 32 filters to find edge features, 3x3 kernel size, 
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(BATCH_SIZE, IMAGE_LENGTH, IMAGE_WIDTH, RGB)),
    #Max Pooling layer with kernel size of 2x2
    layers.MaxPooling2D((2,2)),
    
    #Stacking 4 more of these layers with 64 filters in convolutional layers
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    #Flattening layers into one column vector to feed into feed-forward network
    layers.Flatten(),
    
    #Adding layers to feed-forward neural network
    layers.Dense(64, activation='relu'),
    
    #Prediction layer with 3 neurons for rock, paper, or scissors
    layers.Dense(class_size, activation='softmax')
])

#Building the model
model.build((BATCH_SIZE, IMAGE_LENGTH, IMAGE_WIDTH, RGB))

In [74]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (16, 244, 244, 3)         0         
                                                                 
 sequential_3 (Sequential)   (16, 244, 244, 3)         0         
                                                                 
 conv2d_10 (Conv2D)          (16, 242, 242, 32)        896       
                                                                 
 max_pooling2d_10 (MaxPoolin  (16, 121, 121, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (16, 119, 119, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPoolin  (16, 59, 59, 64)         0         
 g2D)                                                 

In [75]:
#Compiling the model with appropriate optimizers
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']

)

In [77]:
#Training the model
output = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset
)

Epoch 1/50
195/195 [==============================] - 95s 485ms/step - loss: 2.1277 - accuracy: 0.3258 - val_loss: 1.7207 - val_accuracy: 0.4245
Epoch 2/50
195/195 [==============================] - 95s 486ms/step - loss: 2.0716 - accuracy: 0.3474 - val_loss: 1.6643 - val_accuracy: 0.4843
Epoch 3/50
195/195 [==============================] - 95s 488ms/step - loss: 2.0204 - accuracy: 0.3583 - val_loss: 1.5571 - val_accuracy: 0.4615
Epoch 4/50
195/195 [==============================] - 99s 507ms/step - loss: 1.9988 - accuracy: 0.3756 - val_loss: 1.5868 - val_accuracy: 0.4587
Epoch 5/50
195/195 [==============================] - 96s 492ms/step - loss: 1.9973 - accuracy: 0.3778 - val_loss: 1.5126 - val_accuracy: 0.4587
Epoch 6/50
195/195 [==============================] - 97s 498ms/step - loss: 1.9404 - accuracy: 0.3868 - val_loss: 1.4765 - val_accuracy: 0.4786
Epoch 7/50
195/195 [==============================] - 97s 496ms/step - loss: 1.8790 - accuracy: 0.4119 - val_loss: 1.4998 - val_ac

In [99]:
#Saving the Models
pretrained_models.save(r"C:\Users\jakem\OneDrive\Desktop\School\Machine Learning\Final Project\Saved_Model\PretrainedModel.h5")

In [78]:
model.save(r"C:\Users\jakem\OneDrive\Desktop\School\Machine Learning\Final Project\Saved_Model\CustomModel.h5")